In [63]:
print("Hello")

Hello


In [102]:
import os
os.chdir(r"D:\AI\Medi__Chat")

In [103]:
%pwd

'D:\\AI\\Medi__Chat'

In [104]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [105]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="**/*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [106]:
extracted_data = load_pdf(data ='Data/')

In [107]:
extracted_data

[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Untitled document', 'source': 'Data\\napa.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content="Napa  Tablet  \n Pack  Image \nParacetamol \n500  mg  \nBeximco  Pharmaceuticals  Ltd. \nUnit  Price:  ৳  1.20  (51  x  10:  ৳  612.00)  \nStrip  Price: ৳  12.00  \nAlso  available  as:  \n665  mg  (ER  Tablet) 500  mg  (Tablet) 1000  mg  (Tablet) \nAlternate  Brands Innovator's  Monograph     বাংলায়  \x00দ খ ু ন  \nIndications  \nNapa  is  indicated  for  fever,  common  cold  and  influenza,  headache,  toothache,  earache,  bodyache,  \nmyalgia,\n \nneuralgia,\n \ndysmenorrhoea,\n \nsprains,\n \ncolic\n \npain,\n \nback\n \npain,\n \npost-operative\n \npain,\n \npostpartum\n \npain,\n \ninflammatory\n \npain\n \nand\n \npost\n \nvaccination\n \npain\n \nin\n \nchildren.\n \nIt\n \nis\n \nalso\n \nindicated\n \nfor\n \nrheumatic\n \n&\n \nosteoarthritic\n \

In [108]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunks = text_splitter.split_documents(extracted_data)
    return texts_chunks

In [109]:
texts_chunks = text_split(extracted_data)
print(f"Total number of chunks: {len(texts_chunks)}")

Total number of chunks: 41


In [110]:
from langchain.embeddings import HuggingFaceEmbeddings

In [111]:
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [112]:
embeddings= download_huggingface_embeddings()

In [113]:
query_result = embeddings.embed_query("What is the capital of France?")
print("Query result:" ,len(query_result)) 

Query result: 384


In [115]:
query_result


[0.08204810321331024,
 0.036055512726306915,
 -0.00389288691803813,
 -0.004881049506366253,
 0.025651132687926292,
 -0.05714340880513191,
 0.012191588059067726,
 0.004678947385400534,
 0.034949883818626404,
 -0.022421862930059433,
 -0.008005218580365181,
 -0.10935357213020325,
 0.022724738344550133,
 -0.02932084910571575,
 -0.04352204129099846,
 -0.1202412098646164,
 -0.0008486099541187286,
 -0.018150143325328827,
 0.0561295747756958,
 0.003085245843976736,
 0.0023363654036074877,
 -0.016839245334267616,
 0.06362469494342804,
 -0.02366028167307377,
 0.03149349242448807,
 -0.03479795157909393,
 -0.020548852160573006,
 -0.0027910282369703054,
 -0.011038017459213734,
 -0.036126699298620224,
 0.0541410855948925,
 -0.03661712631583214,
 -0.025008630007505417,
 -0.03817038983106613,
 -0.0496036559343338,
 -0.015148119069635868,
 0.02131502516567707,
 -0.012740450911223888,
 0.07670092582702637,
 0.044355735182762146,
 -0.010834883898496628,
 -0.029759937897324562,
 -0.016970494762063026,
 -0

In [116]:
from dotenv import load_dotenv
load_dotenv()

True

In [117]:
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")


In [118]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [119]:

import os
from pinecone import Pinecone, ServerlessSpec

api_key = os.environ.get("PINECONE_API_KEY")

if not api_key:
    raise ValueError("PINECONE_API_KEY not found in environment variables.")
pc = Pinecone(api_key=api_key)

index_name = "medichat"

if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"  
        )
    )

print(f"Index '{index_name}' is ready!")

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '01d3916f89015448a0f1f7840ea7a864', 'date': 'Fri, 06 Jun 2025 09:10:55 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [120]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunks,
    index_name='medichat',
    embedding=embeddings,
)

In [121]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name='medichat',
    embedding=embeddings,
)

In [122]:
docsearch

In [123]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [124]:
retrived_docs = retriver.invoke("What is the Napa?")

In [125]:
retrived_docs

[Document(id='d6f5e4e8-c56a-4e79-9ff8-31efedd60606', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 0.0, 'page_label': '1', 'producer': 'Skia/PDF m138 Google Docs Renderer', 'source': 'Data\\napa.pdf', 'title': 'Untitled document', 'total_pages': 9.0}, page_content="Napa  Tablet  \n Pack  Image \nParacetamol \n500  mg  \nBeximco  Pharmaceuticals  Ltd. \nUnit  Price:  ৳  1.20  (51  x  10:  ৳  612.00)  \nStrip  Price: ৳  12.00  \nAlso  available  as:  \n665  mg  (ER  Tablet) 500  mg  (Tablet) 1000  mg  (Tablet) \nAlternate  Brands Innovator's  Monograph     বাংলায়  \x00দ খ ু ন  \nIndications  \nNapa  is  indicated  for  fever,  common  cold  and  influenza,  headache,  toothache,  earache,  bodyache,  \nmyalgia,\n \nneuralgia,\n \ndysmenorrhoea,\n \nsprains,\n \ncolic\n \npain,"),
 Document(id='308e22bf-cb35-4a0d-887c-cc99f6e118c0', metadata={'creationdate': '', 'creator': 'PyPDF', 'page': 0.0, 'page_label': '1', 'producer': 'Skia/PDF m138 Google Docs Renderer', 'source': 'Dat

In [126]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",                    
    temperature=0,
    max_output_tokens=500,
    google_api_key=GEMINI_API_KEY 
)

In [127]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful medical assistant. "
    "You will be provided with a question and some context from medical documents. "
    "Use the context to answer the question as accurately as possible."
    "{context}\n"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [128]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


question_answer_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(
    retriever=retriver,
    combine_docs_chain=question_answer_chain,
)


In [129]:

import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

API_KEY = GEMINI_API_KEY

print("Testing API Key...")
print(f"API Key length: {len(API_KEY)}")
print(f"API Key starts with 'AIza': {API_KEY.startswith('AIza')}")
try:
    genai.configure(api_key=API_KEY)
    print("\n=== Testing with google.generativeai ===")
    models = list(genai.list_models())
    print(f"Successfully connected! Found {len(models)} models")
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content("Hello, world!")
    print(f"Test response: {response.text[:50]}...")
    
except Exception as e:
    print(f"Direct API test failed: {e}")
try:
    print("\n=== Testing with LangChain ===")
    

    os.environ["GOOGLE_API_KEY"] = API_KEY
    
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0
    )
    
    response = llm.invoke("Say hello")
    print(f"LangChain test successful: {response.content[:50]}...")
    
except Exception as e:
    print(f"LangChain test failed: {e}")
    
    try:
        print("Trying direct API key parameter...")
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            google_api_key=API_KEY
        )
        
        response = llm.invoke("Say hello")
        print(f"Direct API key test successful: {response.content[:50]}...")
        
    except Exception as e2:
        print(f"Direct API key test also failed: {e2}")

print("\n=== Debugging Info ===")
print(f"Environment GOOGLE_API_KEY set: {'GOOGLE_API_KEY' in os.environ}")
if 'GOOGLE_API_KEY' in os.environ:
    env_key = os.environ['GOOGLE_API_KEY']
    print(f"Environment key length: {len(env_key)}")
    print(f"Environment key starts with 'AIza': {env_key.startswith('AIza')}")

Testing API Key...
API Key length: 39
API Key starts with 'AIza': True

=== Testing with google.generativeai ===
Successfully connected! Found 58 models
Test response: Hello, world!
...

=== Testing with LangChain ===
LangChain test successful: Hello!...

=== Debugging Info ===
Environment GOOGLE_API_KEY set: True
Environment key length: 39
Environment key starts with 'AIza': True


In [130]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=API_KEY  
)

response = llm.invoke("Say hello")
print(f"LangChain test successful: {response.content}")


LangChain test successful: Hello!


In [132]:
import time


time.sleep(1)  
response = rag_chain.invoke({"input": "side effect of Napa"})

response_text = response['answer']


prefix = "Based on the provided text, "
if response_text.startswith(prefix):
    response_text = response_text[len(prefix):]

print(response_text)


the side effects of Napa are usually mild.  However,  hematological reactions including thrombocytopenia, leucopenia, pancytopenia, neutropenia, and agranulocytosis have been reported.  Pancreatitis, skin rashes, and other allergic reactions also occur occasionally.
